# Chatbot
In this tutorial, we'll be designing a chatbot with the capability to retain information from previous prompts and responses, enabling it to maintain context throughout the conversation. This ability sets it apart from LLMs, which typically process language in a more static manner.


---
## 1.&nbsp; Installations and Settings 🛠️
Let's replicate the process of downloading and installing the necessary libraries and model, just like we did in the previous LLM notebook. If you had any specific settings in the first notebook, such as using a CPU instead of a GPU, maintain those same configurations here.

Since we'll be continuing our work on Colab with a GPU, let's activate it by navigating to `Edit > Notebook Settings`. Select `GPU` and click `Save`.

In [ ]:
!pip3 install -qqq langchain --progress-bar off
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in resolve
    self._a

---
## 2.&nbsp; Setting up your LLM 🧠

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path = "/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens = 2000,
               temperature = 0.1,
               top_p = 1,
               n_gpu_layers = -1)

### 2.1.&nbsp; Test your LLM

In [ ]:
answer_1 = llm.invoke("Write a poem about data science.")
print(answer_1)

---
## 3.&nbsp; Making a chatbot 💬
To transform a basic LLM into a chatbot, we'll need to infuse it with additional functionalities: prompts, memory, and chains.

**Prompts** are like the instructions you give the chatbot to tell it what to do. Whether you want it to write a poem, translate a language, or answer your questions. They provide the context and purpose for its responses.

**Memory** is like the chatbot's brain. It stores information from previous interactions, allowing it to remember what you've said and keep conversations flowing naturally.

The **chain** is like a road map that guides the conversation along the right path. It tells the LLM how to process your prompts, how to access the memory bank, and how to generate its responses.

In essence, prompts provide the direction, memory retains the context, and chains orchestrate the interactions.

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

### Prompt ###
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Keep your answers very short, succinct, and to the point."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

### Memory ###
# We `return_messages=True` to fit into the MessagesPlaceholder
# `"chat_history"` must align with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key = "chat_history",
                                  return_messages=True)

### Chain ###
conversation = LLMChain(llm = llm,
                        prompt = prompt,
                        verbose = True,
                        memory = memory)

### 3.1.&nbsp; Test your chatbot

In [ ]:
conversation({"question": "hi"})

In [ ]:
conversation({"question": "Translate this sentence from English to French: I love programming."})

In [ ]:
conversation({"question": "Now translate the sentence to German."})

In [ ]:
conversation({"question": "Which contains more characters, the French translation or the German?"})

---
## 4.&nbsp; Challenge 😀
1. Play around with writing new prompts.
  * Try having an empty prompt, what does it do to the output?
  * Try having a funny prompt.
  * Try having a long, precise prompt.
  * Try all different kinds of prompts.
2. Try out the different types of memory. What effect does this have on the type of conversations you have? Slower or faster? Better or worse accuracy? Forgetting things or super precise?
  * [ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window) - Only a set number of k remembered - e.g. last 6 inputs
  * [ConversationSummaryMemory](https://python.langchain.com/docs/modules/memory/types/summary) - summarises the history to save space
  * [ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer) - summarises and restricted by token length
3. Try different LLMs with different types of prompts and memory. Which combination works best for you? Why?